In [44]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [45]:
# Data Directory and Categories
data_dir = 'Severity_Dataset'
categories = ['Level 0', 'Level 1', 'Level 2', 'Level 3']
img_height, img_width = 224, 224
batch_size = 16

In [46]:
# Function to Load Images and Labels
def load_data(data_dir, categories):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_width, img_height))
            images.append(img)
            labels.append(categories.index(category))
    return np.array(images), np.array(labels)

In [47]:
import os

data_dir = 'Severity_Dataset'
print("Does data directory exist?", os.path.exists(data_dir))

for category in categories:
    category_path = os.path.join(data_dir, category)
    print(f"Checking: {category_path} -> Exists? {os.path.exists(category_path)}")

Does data directory exist? True
Checking: Severity_Dataset\Level 0 -> Exists? True
Checking: Severity_Dataset\Level 1 -> Exists? True
Checking: Severity_Dataset\Level 2 -> Exists? True
Checking: Severity_Dataset\Level 3 -> Exists? True


In [48]:
# Load and Normalize Data
images, labels = load_data(data_dir, categories)
images = images / 255.0

In [49]:
# Train-Test Split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

In [50]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [51]:
# EfficientNetB0 Model with Additional Layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = True  # Enable fine-tuning

In [52]:
model = Sequential([
    base_model,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])

In [53]:
# Compile Model with Optimized Learning Rate
model.compile(optimizer=Adam(learning_rate=0.00005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
# Train Model with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    validation_data=(X_val, y_val),
                    epochs=50,
                    callbacks=[early_stopping],
                    verbose=1)

d:\GitHub\Acne_severe\model\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.2378 - loss: 1.4383 - val_accuracy: 0.2564 - val_loss: 1.3875
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 38s 974ms/step - accuracy: 0.3060 - loss: 1.3686 - val_accuracy: 0.2564 - val_loss: 1.3903
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 39s 985ms/step - accuracy: 0.3770 - loss: 1.3017 - val_accuracy: 0.2500 - val_loss: 1.3926
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.4179 - loss: 1.2748 - val_accuracy: 0.2500 - val_loss: 1.4060
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.3782 - loss: 1.2737 - val_accuracy: 0.2500 - val_loss: 1.4393
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4693 - loss: 1.1840 - val_accuracy: 0.2500 - val_loss: 1.4820
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4756 - loss: 1.1489 - val_accuracy: 0.2436 - val_loss: 1.5528
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5202 - loss: 1.0953 - val_accuracy: 0.2436 - val

In [55]:
# Evaluate Model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 376ms/step - accuracy: 0.6420 - loss: 0.8780
Validation Accuracy: 0.6603


In [56]:
model.save("acne_severity_model.h5")